In [1]:
import pandas as pd
import numpy as np
import torch

In [2]:
# InvoiceNo: 주문 번호
# StockCode: 상품 코드
# Description: 상품 설명
# Quantity: 상품 수량
# InvoiceDate: 주문 날짜
# UnitPrice: 상품 단가
# CustomerID: 고객 ID
# Country: 국가

In [3]:
# online_retail.csv 파일 불러오기
df = pd.read_csv(f"data/online_retail.csv")
df = df.dropna()
df.shape

(406829, 8)

In [4]:
df['StockCodeNo'] = df['StockCode'].astype("category").cat.codes
# 상품 코드와 고객 ID로 이루어진 새로운 데이터프레임 생성
df_new = df[['StockCodeNo', 'CustomerID', 'Quantity']]

# NaN 값을 가지는 행 제거
df_new = df_new.dropna()

# 고객 ID와 상품 코드를 정수형으로 변환
df_new['user'] = df_new['CustomerID'].astype("category").cat.codes
df_new['item'] = df_new['StockCodeNo'].astype("category").cat.codes

MF 클래스는 nn.Module을 상속받아서 정의되며, n_users와 n_items는 각각 사용자와 아이템의 수이며, n_factors는 잠재 요인(latent factor)의 수입니다. 모델의 학습을 통해 잠재 요인이 학습됩니다.

nn.Embedding은 입력 텐서에 대해 임베딩을 수행하는 클래스입니다. 여기서는 각 사용자와 아이템을 임베딩합니다. user_factors와 item_factors는 각각 사용자와 아이템의 잠재 요인을 나타내는 행렬입니다. 임베딩된 사용자와 아이템 벡터는 forward 함수에서 곱해져서 예측 평점을 계산합니다.

forward 함수에서는 각 사용자와 아이템의 임베딩 벡터를 가져와서 내적(dot product)을 계산합니다. 내적의 결과는 각 사용자와 아이템의 잠재 요인 값들을 조합한 예측 평점을 나타냅니다. 따라서 모델의 출력은 사용자-아이템 쌍에 대한 예측 평점의 벡터가 됩니다.

In [5]:
import torch
from torch import nn

In [6]:
df_ratings = df_new.groupby(["user", "item"])["Quantity"].mean().reset_index()
df_ratings.head()

,user,item,Quantity
0,0,2001,0.0
1,1,25,24.0
2,1,87,36.0
3,1,130,6.0
4,1,167,10.0


In [7]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(df_ratings[['user', 'item', 'Quantity']], test_size=0.2)

In [8]:
train_df.head()

,user,item,Quantity
92201,1534,2777,2.0
45879,752,2109,12.0
266680,4348,1105,1.0
231193,3758,863,288.0
218203,3562,412,12.0


In [9]:
df_ratings.isnull().sum()

user        0
item        0
Quantity    0
dtype: int64

In [10]:
from sklearn.model_selection import train_test_split

df_ratings[["user", "item"]] = torch.LongTensor(df_ratings[["user", "item"]].values)
df_ratings[["Quantity"]] = torch.LongTensor(df_ratings[["Quantity"]].values)

# 데이터셋을 train set과 test set으로 나눔
train_df, val_df = train_test_split(df_ratings, test_size=0.2)

# 사용자 수와 상품 수 계산
n_users = df_ratings['user'].nunique()
n_items = df_ratings['item'].nunique()

# DataLoader
train_loader = torch.utils.data.DataLoader(train_data, batch_size=256, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=256, shuffle=False)

train_data

NameError: name 'train_data' is not defined

In [ ]:
print(n_users, n_items)

In [ ]:
class MF(nn.Module):
    def __init__(self, n_users, n_items, n_factors=20):
        super(MF, self).__init__()
        self.user_factors = nn.Embedding(n_users, n_factors)
        self.item_factors = nn.Embedding(n_items, n_factors)

    def forward(self, user, item):
        user_embedding = self.user_factors(user)
        item_embedding = self.item_factors(item)
        return (user_embedding * item_embedding).sum(1)

model = MF(n_users, n_items, n_factors=20)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-5)

In [ ]:
for epoch in range(3):
    for train_batch in train_loader.dataset[:3]:
        user, item, rating = train_batch
        optimizer.zero_grad()
        output = model(user.view(-1), item.view(-1))
        print("output", output, "rating", rating.unsqueeze(0))
        loss = criterion(output, rating)
        print(output, loss)
        loss.backward()
        optimizer.step()

    with torch.no_grad():
        val_loss = 0.0
        for val_batch in val_loader.dataset:
            user, item, rating = val_batch
            output = model(user, item)
            val_loss += criterion(output, rating).item() * len(user)
        val_loss /= len(val_df)
        print('epoch: {}, validation RMSE loss: {:.4f}'.format(epoch+1, val_loss**0.5))

In [ ]:
# 각 제품의 인덱스를 제품 코드로 매핑
idx_to_item = {i: item for item, i in item_to_idx.items()}

# 특정 사용자에게 추천할 상위 10개의 제품 출력
user_idx = 0
user_items = set(train_df[train_df['user'] == user_idx]['item'])
scores = model(torch.LongTensor([user_idx]*len(item_to_idx)), torch.LongTensor(list(item_to_idx.values()))).detach().numpy()
item_indices = list(range(len(item_to_idx)))
item_scores = list(zip(item_indices, scores))
item_scores = sorted(item_scores, key=lambda x: x[1], reverse=True)
recommended_items = []
for item_idx, score in item_scores:
    item_code = idx_to_item[item_idx]
    if item_code not in user_items:
        recommended_items.append(item_code)
    if len(recommended_items) >= 10:
        break
print('Recommended items for user {}:'.format(user_idx))
print(recommended_items)
